## Kavir_RAG
Retrieval-Augmented Generation
<div style="direction: rtl; white-space: pre-wrap; line-height: 1.5;">
Retrieval = بازیابی (درست مثل سرچ کردن و پیدا کردن یک متن مرتبط از میان کلی سند)
Augmented = تقویت‌شده / غنی‌شده (یعنی اطلاعات جدیدی به چیزی اضافه شده)
Generation = تولید متن (کاری که مدل زبانی انجام می‌ده: جمله‌سازی، پاسخ‌دهی)
</div>

<div style="direction: rtl;">


# تعریف پروژه (گام ۰: صورت‌مسئله و برنامهٔ کار)

## نام‌ها

* **نام پروژه (repo):** `Kavir-RAG`  *(کویر: محلی، مستقل، خلوت)*
* **نام نوت‌بوک اصلی:** `Kavir_RAG_MVP.ipynb`

## هدف فنی

یک **سرور RAG کاملاً لوکال و آفلاین** برای تحلیل اسناد چندفرمت (PDF, DOCX, XLSX/CSV, TXT, code snippets, IPYNB) با:

* ایندکس‌سازی (FAISS/Chroma) + امبدینگ چندزبانه (fa/en)
* بازیابی معنایی + (اختیاری) ریرنکر
* پاسخ LLM لوکال با **ارجاع به منبع**
* API مبتنی بر **FastAPI** (endpoints: `/upload`, `/query`, `/chat`, `/reindex`) با **توکن ساده**
* اجرا روی i5 / 8GB RAM (CPU-only؛ 16GB روان‌تر)

## پیکرهٔ فنی پیشنهادی

* **LLM**: Qwen2-7B-Instruct (Q4\_0 via Ollama/llama.cpp) – کم‌مصرف و چندزبانه
* **Embedding**: `intfloat/multilingual-e5-small` (CPU-friendly)
* **Vector DB**: FAISS (IndexFlatIP + نُرمال‌سازی → cosine)
* **Re-rank (اختیاری)**: `bge-reranker-base` (CPU)
* **Parserها**: PyMuPDF (PDF)، python-docx، pandas (CSV/XLSX)، nbformat (ipynb)، Tesseract (OCR در صورت نیاز)
* **API**: FastAPI + Uvicorn
* **UI سبک (اختیاری)**: Streamlit (یا React ساده)
* **امنیت LAN**: توکن ساده + (اختیاری) Nginx/Caddy با TLS داخلی


## خروجی‌های نسخهٔ پایه (MVP)

* Endpointهای `/upload`, `/query`, `/reindex` (و `/chat` سبک)
* بازیابی top-k با citation
* ذخیرهٔ ایندکس روی دیسک و بارگذاری مجدد
* اجرای کامل روی یک ماشین بدون اینترنت (بعد از دانلود اولیهٔ مدل‌ها)

## زمان‌بندی و هزینه (الگوی آپ‌ورک)

* **MVP (فقط API+RAG)**: \~ **۱ روز کاری**
  شکست به تسک‌ها: تعاریف و اسکلت (۱س) • نصب/مدل (۲س) • ingest+index (۲–۳س) • RAG+API (۲س) • تست سریع (۱س).
* **کامل‌تر (UI، امنیت LAN، بکاپ، ریرنکر، هیبرید سرچ)**: **۲–۴ روز کاری**
* **تخمین هزینه** (فرضِ فریلنسری استاندارد):

  * MVP: **۲۰۰–۳۰۰ دلار**
  * کامل‌تر: **۵۰۰–۸۰۰ دلار**
* **آپشن توسعهٔ حرفه‌ای (۳–۷ روز اضافه، +۷۰۰ تا +۱۵۰۰ دلار بسته به دامنه):**

  * هیبرید سرچ (BM25 + dense) و تیونینگ ریتریور
  * ریرنکر پیش‌فرض + A/B ارزیابی پاسخ‌ها
  * Session memory برای چت چندمرحله‌ای
  * Docker Compose + Nginx/Caddy + TLS داخلی
  * Role-based tokens، لاگ و Auditing
  * Sandbox خواندن امنِ ipynb (بدون اجرای کُد کاربر)
  * تست‌های واحد/تکاملی + بنچمارک کوچک کیفیت RAG
  * اسکریپت پشتیبان‌گیری ایندکس + بازیابی

## ریسک‌ها و قیود

* 8GB RAM کفِ قابل اجراست؛ زمان ساخت امبدینگ اولیه کندتر است.
* فایل‌های تصویریِ فرمول نیازمند OCR هستند و ممکن است کیفیت استخراج متغیر باشد.
* بارِ نخست نیاز به دانلود مدل‌ها دارد؛ سپس کاملاً آفلاین.

---

# گام بعدی (مرحله ۱: اسکلت پروژه و آماده‌سازی محیط)

در این مرحله فقط **ساختار پوشه‌ها** و یک **venv سبک** ایجاد می‌کنیم و فهرست **dependencies** را می‌نویسیم (بدون نصب سنگین مدل‌ها هنوز).
طبق «پرامپتِ قدم»، **هیچ کُدی** تا وقتی تأیید ندهی نمایش نمی‌دهم.


</div>

In [11]:
'''Kavir-RAG/
  data/
    raw/           # raw uploaded files
    ocr/           # OCR output (if needed)
    index/         # FAISS index stored on disk
  models/          # cached models (optional)
  app/
    api.py         # FastAPI endpoints
    rag_core.py    # retrieval + prompt composition + generation
    ingest.py      # file parsers + chunking + embeddings
    auth.py        # simple token-based authentication
    utils.py       # helper functions
  notebooks/
    Kavir_RAG_MVP.ipynb
  configs/
    settings.yaml  # configuration file
  tests/           # unit/integration tests
  README.md        # project documentation
  '''


'Kavir-RAG/\n  data/\n    raw/           # raw uploaded files\n    ocr/           # OCR output (if needed)\n    index/         # FAISS index stored on disk\n  models/          # cached models (optional)\n  app/\n    api.py         # FastAPI endpoints\n    rag_core.py    # retrieval + prompt composition + generation\n    ingest.py      # file parsers + chunking + embeddings\n    auth.py        # simple token-based authentication\n    utils.py       # helper functions\n  notebooks/\n    Kavir_RAG_MVP.ipynb\n  configs/\n    settings.yaml  # configuration file\n  tests/           # unit/integration tests\n  README.md        # project documentation\n  '

In [12]:
!touch Dockerfile
!touch requirements.txt
!mkdir -p src
!touch src/main.py

In [13]:
pip install fastapi uvicorn

Note: you may need to restart the kernel to use updated packages.


<div style="direction: rtl; white-space: pre-wrap; line-height: 1;">

### 2. FAISS دقیقاً چیکار می‌کنه؟

* **FAISS (Facebook AI Similarity Search)** یه کتابخونه است برای **جستجوی سریع در وکتورها**.
* وقتی ما متن‌ها رو به امبدینگ (بردار) تبدیل کنیم، FAISS کمک می‌کنه سریع‌ترین متن‌های مشابه رو پیدا کنیم.
* خلاصه: مغز قسمت «Retrieve» در RAG هست.

---

### 3. الان کجاییم؟

ما Step 1 رو اینجوری پیش بردیم:

* ✅ ساخت پوشه‌ها و فایل‌ها
* ✅ نوشتن و تست `main.py` (FastAPI)
* ✅ ساخت و نصب `requirements.txt` (بدون faiss روی مک)

🔜 مرحله بعدی همون Step 1:

* نوشتن **Dockerfile** → تا محیط لینوکسی بسازیم و اونجا `faiss-cpu` هم درست نصب بشه.


</div>

<div style="direction: rtl; white-space: pre-wrap; line-height: 1;">
# مرحله ۱: آماده‌سازی محیط و اسکلت پروژه (Kavir-RAG)

## هدف
ایجاد اسکلت اولیه پروژه با FastAPI و Docker برای اجرای سرور محلی.

## اقدامات انجام‌شده
- ساخت پوشه‌ها و فایل‌های پایه (`src/main.py`, `requirements.txt`, `Dockerfile`)
- نوشتن کد تستی FastAPI با endpoint `/ping`
- نصب پکیج‌های پایه با `requirements.txt` (بدون faiss روی مک)
- نوشتن Dockerfile و Build ایمیج با `python:3.11-slim`
- اجرای کانتینر و تست `/ping` و `/docs` از داخل Docker
- افزودن `.gitignore` برای جلوگیری از push فایل‌های حجیم (مدل‌ها، دیتا، ایندکس)

## وضعیت فعلی
✅ سرور FastAPI داخل کانتینر Docker به‌درستی اجرا شد و تست اولیه موفق بود.

## گام بعدی (مرحله ۲)
- ماژول ingestion برای خواندن فایل‌های PDF, Word, CSV, Notebook
- chunking متن
- تولید امبدینگ (multilingual-e5-small)
- ذخیره در ایندکس FAISS

</div>

<div style="direction: rtl; white-space: pre-wrap; line-height: 1.5;">
سلول 1 — ساخت پوشه‌ها و فایل‌های نمونه (صرفاً پایتون)
</div>

In [14]:
import os, json, textwrap, pathlib

# ریشه پروژه را همین پوشه نوت‌بوک فرض می‌کنیم
BASE = pathlib.Path(".").resolve()
(DATA_RAW, DATA_CHUNKS, SRC) = (BASE/"data"/"raw", BASE/"data"/"chunks", BASE/"src")
for p in (DATA_RAW, DATA_CHUNKS, SRC):
    p.mkdir(parents=True, exist_ok=True)

# نمونه فایل متنی (اعداد انگلیسی تا مشکلی پیش نیاد)
sample_txt = """این یک متن آزمایشی برای تست چانکینگ است.
هدف: تقسیم متن به قطعات 800 کاراکتری با همپوشانی 100.
"""
(DATA_RAW/"sample.txt").write_text(sample_txt, encoding="utf-8")

# نمونه CSV
csv_txt = "region,price,year,manufacturer,model\nseattle,12000,2012,toyota,corolla\nla,8000,2010,honda,civic\n"
(DATA_RAW/"cars.csv").write_text(csv_txt, encoding="utf-8")

print("ساخته شد:", DATA_RAW)
print([p.name for p in DATA_RAW.iterdir()])


ساخته شد: /Users/macbookpro/AMIR_DATA/00_Project/VSCode__project/1_Machine_learning/with_AI/ML_Models/simulation_upwork/Kavir_RAG(Retrieval Augmented Generation)/data/raw
['cars.csv', 'sample.txt']


<div style="direction: rtl; white-space: pre-wrap; line-height: 1.5;">
سلول 2 — اجرای ingestion.py از داخل نوت‌بوک با همان Python فعلی
</div>

In [16]:
import sys, subprocess, pathlib
REQ = pathlib.Path("requirements.txt").resolve()
print("Using interpreter:", sys.executable)
print("Installing from:", REQ)
subprocess.check_call([sys.executable, "-m", "pip", "install", "-r", str(REQ)])


Using interpreter: /usr/local/bin/python3
Installing from: /Users/macbookpro/AMIR_DATA/00_Project/VSCode__project/1_Machine_learning/with_AI/ML_Models/simulation_upwork/Kavir_RAG(Retrieval Augmented Generation)/requirements.txt
  Using cached pymupdf-1.26.4-cp39-abi3-macosx_10_9_x86_64.whl.metadata (3.4 kB)
Using cached pymupdf-1.26.4-cp39-abi3-macosx_10_9_x86_64.whl (23.1 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [pdf2image]/3 [pymupdf]


0

In [17]:
import sys, subprocess, shlex, pathlib

PY = shlex.quote(sys.executable)  # مسير همين پایتون (مثلاً /usr/bin/python3.11)
cmd = (
    f'{PY} src/ingestion.py '
    f'--path {shlex.quote(str(DATA_RAW))} '
    f'--out {shlex.quote(str(DATA_CHUNKS))} '
    f'--chunk-size 800 --overlap 100 --preview'
)
print("RUN:", cmd)

proc = subprocess.run(cmd, shell=True, capture_output=True, text=True)
print("STDOUT:\n", proc.stdout)
print("STDERR:\n", proc.stderr)
if proc.returncode != 0:
    raise RuntimeError(f"ingestion.py failed with code {proc.returncode}")


RUN: /usr/local/bin/python3 src/ingestion.py --path '/Users/macbookpro/AMIR_DATA/00_Project/VSCode__project/1_Machine_learning/with_AI/ML_Models/simulation_upwork/Kavir_RAG(Retrieval Augmented Generation)/data/raw' --out '/Users/macbookpro/AMIR_DATA/00_Project/VSCode__project/1_Machine_learning/with_AI/ML_Models/simulation_upwork/Kavir_RAG(Retrieval Augmented Generation)/data/chunks' --chunk-size 800 --overlap 100 --preview
STDOUT:
 ✅ Ingested: /Users/macbookpro/AMIR_DATA/00_Project/VSCode__project/1_Machine_learning/with_AI/ML_Models/simulation_upwork/Kavir_RAG(Retrieval Augmented Generation)/data/raw/cars.csv
   chars=96  chunks=1  -> /Users/macbookpro/AMIR_DATA/00_Project/VSCode__project/1_Machine_learning/with_AI/ML_Models/simulation_upwork/Kavir_RAG(Retrieval Augmented Generation)/data/chunks/cars__030047719b5e.jsonl
   preview: region,price,year,manufacturer,model seattle,12000,2012,toyota,corolla la,8000,2010,honda,civic  ...
✅ Ingested: /Users/macbookpro/AMIR_DATA/00_Project/VS

سلول 3 — بررسی خروجی‌ها (خواندن JSONL با پایتون)

In [18]:
from glob import glob
import json, pathlib
CHUNKS_DIR = pathlib.Path("data/chunks")

files = sorted(glob(str(CHUNKS_DIR/"*.jsonl")))
print("Found:", len(files), "chunk files")
for fp in files:
    print("—", pathlib.Path(fp).name)

if not files:
    raise SystemExit("هیچ فایل چانکی پیدا نشد. اول ingestion رو اجرا کن.")

total_lines = 0
for fp in files:
    print("\n== Preview:", pathlib.Path(fp).name, "==")
    with open(fp, "r", encoding="utf-8") as f:
        # سه خط اول برای نمونه
        for i in range(3):
            line = f.readline()
            if not line: break
            obj = json.loads(line)
            total_lines += 1
            print(f"[{i+1}] chunk_index={obj['chunk_index']}  range={obj['char_range']}  src={obj['metadata']['source_name']}")
            print("   text:", obj["text"][:120].replace("\n", " "), "...")
print("\nTotal previewed lines:", total_lines)


Found: 2 chunk files
— cars__030047719b5e.jsonl
— sample__54e5a362dddd.jsonl

== Preview: cars__030047719b5e.jsonl ==
[1] chunk_index=0  range=[0, 96]  src=cars.csv
   text: region,price,year,manufacturer,model seattle,12000,2012,toyota,corolla la,8000,2010,honda,civic  ...

== Preview: sample__54e5a362dddd.jsonl ==
[1] chunk_index=0  range=[0, 95]  src=sample.txt
   text: این یک متن آزمایشی برای تست چانکینگ است. هدف: تقسیم متن به قطعات 800 کاراکتری با همپوشانی 100.  ...

Total previewed lines: 2


<div style="direction: rtl; white-space: pre-wrap; line-height: 1;">
باشه امیر 👌 یک مرور جامع از پروژه RAG لوکال تا اینجا (طبق تعریف اولیه‌ای که گذاشتی: ingest → embed → index → retrieval → API)

---

# 📝 خلاصه پیشرفت پروژه RAG (تا اینجا)

## ✅ کارهای انجام‌شده

### 1. ساختار و محیط

* پوشه‌بندی پروژه (`src/`, `data/`, `requirements.txt`, `Dockerfile`) ساخته شد.
* `.gitignore` تنظیم شد → دیتای سنگین (`data/`, `.cache/`) و کش HuggingFace نادیده گرفته می‌شوند.
* Dockerfile نهایی شد:

  * بیس: `python:3.11-slim`
  * نصب پکیج‌های سیستمی (Tesseract, Poppler)
  * نصب پکیج‌های پایتون (FastAPI, torch (CPU), faiss-cpu, sentence-transformers, pandas, …)
  * اضافه شدن `ENV HF_HOME=/root/.cache/huggingface` برای مدیریت کش
  * CMD → اجرای `uvicorn` با یک ورکر

### 2. تست محیط و کتابخانه‌ها

* اجرای کانتینر و تست نصب:

  * Torch (CPU-only) ✔
  * FAISS ✔
  * Sentence-Transformers ✔
* مدل `intfloat/multilingual-e5-small` یک‌بار لود شد، کش پایدار روی میزبان تنظیم شد (`.cache/hf`).

### 3. Ingestion + Chunking

* `src/ingestion.py` نوشته شد:

  * پشتیبانی از PDF, DOCX, TXT, CSV
  * چانکینگ متنی (JSONL خروجی → `data/chunks/`)
* تست شد (روی ۲ فایل نمونه) → خروجی چانک OK.

### 4. Embedding + Indexing

* `src/embeddings.py` نوشته شد:

  * چانک‌ها خوانده شد.
  * امبدینگ‌ها با `multilingual-e5-small` ساخته شد.
  * L2 normalize + ذخیره در FAISS (IndexFlatIP)
  * ذخیره متادیتا (`meta.jsonl`) و ایندکس (`faiss.index`) در `data/index/`.
* تست اجرا شد → `Index size: 2 vectors of dim 384`.

### 5. Retrieval

* `src/search.py` نوشته شد:

  * کوئری کاربر امبد شد.
  * FAISS جست‌وجو کرد.
  * متادیتا و متن چانک‌های برتر چاپ شد.
* تست روی کوئری → نتایج درست برگشت.

### 6. API (FastAPI)

* `src/main.py` به‌روزرسانی شد:

  * `/ping` (health check)
  * `/query` (retrieval endpoint)
* تست Swagger (`/docs`) انجام شد → کوئری و نتایج OK.

---

## 🟡 کارهایی که هنوز مانده

### مرحله‌های اصلی پروژه

1. **بهبود اسکیما چانک‌ها**

   * اضافه کردن فیلدهای `id`, `doc_id`, `chunk_idx`, `start`, `end` به خروجی ingestion.
   * اجرای دوباره ingestion + embeddings → ایندکس کامل‌تر.

2. **Re-chunk & Re-index**

   * بعد از اصلاح اسکیما، تمام فایل‌ها دوباره چانک شوند.
   * ایندکس جدید ساخته شود.

3. **بهبود API**

   * `/query`: خروجی تمیزتر (id, doc\_id, text\_preview, score).
   * اضافه کردن `/upload`: آپلود فایل (PDF, DOCX, …) و اجرای ingestion → آپدیت ایندکس.
   * اضافه کردن `/reindex`: ساخت دوباره ایندکس از صفر.
   * اختیاری: `/chat`: رپ کردن retrieval + LLM (Qwen2-7B-Instruct q4).

4. **اتصال LLM**

   * بعد از retrieval، کوئری + context به LLM داده شود.
   * مدل هدف: **Qwen2-7B-Instruct (q4)**.
   * اجرا روی CPU (i5 / 8GB) → نیاز به مدیریت حافظه و batch کوچک.

5. **UI اختیاری**

   * Streamlit یا React برای نمایش نتایج و تعامل ساده‌تر.

---

## 🔮 جمع‌بندی

* تا اینجا **کل مسیر ingest → embed → index → retrieval → API** برای MVP تست شد و جواب داده.
* پروژه الان یک **RAG پایه** داره که میشه رویش توسعه داد.
* کار اصلی مونده: **اضافه کردن LLM به retrieval (مرحله پاسخ‌دهی)** و **endpointهای تکمیلی** برای ingestion/reindex/upload.



</div>

<div style="direction: rtl; white-space: pre-wrap; line-height: 1;">
عالیه امیر 👌

📌 خلاصهٔ این گام:

* `embeddings.py` با موفقیت اجرا شد.
* از روی **۴ چانک** امبدینگ ساخته شد (هر بردار 384 بعدی).
* ایندکس FAISS (`faiss.index`) و متادیتا (`meta.jsonl`) ساخته شد.
* بکاپ‌های قدیمی هم نگه داشته شدند.
* هم داخل کانتینر و هم روی مک در مسیر `data/index/` قابل دیدن هستند. ✅

---

### وضعیت پروژه RAG تا اینجا

1. **Ingestion** → `chunks.jsonl` و `manifest.jsonl` → آماده
2. **Embedding + Indexing** → `faiss.index` و `meta.jsonl` → آماده

---

### 🟡 گام بعدی پیشنهادی

اضافه‌کردن endpoint `/query` در `search.py` یا `main.py` برای این کار:

* گرفتن query از کاربر
* ساخت embedding همان query
* سرچ در FAISS
* برگرداندن: `id`, `doc_id`, `chunk_idx`, `score`, `text_preview`

---

👉 می‌خوای مرحله بعد بریم سراغ ساخت `/query` endpoint؟

</div>